In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("files/apostila.pdf")

docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, separators=['\n\n', '\n', '.', ' ', ''])
# Normalize text by removing repeated punctuation patterns
normalized_docs = []
for doc in docs:
    # Remove repeated dots, dashes and spaces
    cleaned_text = doc.page_content
    cleaned_text = ' '.join(cleaned_text.split())  # Remove extra whitespace
    cleaned_text = cleaned_text.replace('....', '.')
    cleaned_text = cleaned_text.replace('...', '.')
    cleaned_text = cleaned_text.replace('..', '')
    cleaned_text = cleaned_text.replace('----', '-')
    cleaned_text = cleaned_text.replace('---', '-')
    cleaned_text = cleaned_text.replace('--', '-')
    
    # Create new document with cleaned text
    doc.page_content = cleaned_text
    normalized_docs.append(doc)

# Split the normalized documents
chunks = text_splitter.split_documents(normalized_docs)

chunks


[Document(page_content='INTRODUÇÃO À PROGRAMAÇÃO COM PYTHON Programa de Educação Tutorial Grupo PET - ADS IFSP - Câmpus São Carlos', metadata={'source': 'files/apostila.pdf', 'page': 0}),
 Document(page_content='Sumário PREFÁCIO     1 1. INTRODUÇÃO    . 2 1.1 Característi cas da linguagem Python    2 1.2 Instalação do interpretador Python    2 2. VARIÁVEIS    . 4 3. STRINGS    . 6 3.1 Concatenação d e strings    6 3.2 Manipulação de strings    7 3.3 Fatiamento de strings   . 8 3.4 Exercícios: strings     8 4. NÚMEROS    . 9 4.1 Operadores numéricos    9 4.2 Exercícios: números    9 5. LISTAS     10 5.1 Funções para manipulação de listas   . 10 5.2 Operações com listas   . 11 5.3 Fatiamento de listas   . 11 5.4 Criação de listas com range ( )    12 5.5 Exercícios: listas    . 12 6. TUPLAS     13 7. DICIONÁRIOS    . 13 7.1 Operações em dicionários   . 14 7.2 Exercícios: dicionários    14 8. BIBLIOTECAS     15 9. ESTRUTURAS DE DECISÃO   . 15 9.1 Estrutura if    . 16 9.2 Estrutura ifelse  

In [8]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

## Banco vetorial

from langchain_chroma import Chroma

dir = 'db/chroma_db_vectorstore'

vector_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=dir,
)


In [ ]:
ask = "Principais métodos para manipulação de strings"

docs = vector_store.similarity_search(ask, k=5)

for doc in docs:
    print(doc.page_content)
    print(doc.metadata)
    print('-'*100)


{'page': 9, 'source': 'files/apostila.pdf'}
----------------------------------------------------------------------------------------------------
{'page': 10, 'source': 'files/apostila.pdf'}
----------------------------------------------------------------------------------------------------
{'page': 8, 'source': 'files/apostila.pdf'}
----------------------------------------------------------------------------------------------------
{'page': 1, 'source': 'files/apostila.pdf'}
----------------------------------------------------------------------------------------------------
{'page': 9, 'source': 'files/apostila.pdf'}
----------------------------------------------------------------------------------------------------


In [9]:
vector_store._collection.count()

46